# Bsky integration

In [ ]:
//| export

import { Agent } from "@atproto/api";
import { z } from "zod";

const profileSchema = z.object({
  did: z.string(),
  handle: z.string(),
  displayName: z.string().optional(),
  avatar: z.string().optional(),
  labels: z.array(z.string()),
  createdAt: z.string(),
  description: z.string().optional(),
  banner: z.string().optional(),
  followersCount: z.number(),
  followsCount: z.number(),
  postsCount: z.number(),
});

export type Profile = z.infer<typeof profileSchema>;

export const getProfile = async (did: string): Promise<Profile> => {
  const { data } = await (new Agent("https://public.api.bsky.app/xrpc"))
    .getProfile({ actor: did });
  return profileSchema.parse(data);
};

In [ ]:
// await getProfile("did:plc:ubdeopbbkbgedccgbum7dhsh");

// CDN for the image
// https://github.com/notjuliet/pdsls/blob/c86372402cc5cb78c72277938adc2912b8100a85/src/components/json.tsx#L114-L129

## Richtext processing

Check out
https://github.com/mary-ext/skeetdeck/tree/aa0cb74c0ace489b79d2671c4b9e740ec21623c7/app/api/richtext

In [ ]:
//| export

import { RichText } from "@atproto/api";

export const getRichText = async (text: string): Promise<RichText> => {
  const rt = new RichText({ text });
  await rt.detectFacets(new Agent("https://public.api.bsky.app/xrpc"));
  return rt;
};

// console.log({
//   $type: "app.bsky.feed.post",
//   text: rt.text,
//   facets: rt.facets,
//   createdAt: new Date().toISOString(),
// });

In [ ]:
import { assert, assertEquals } from "asserts";

Deno.test("getRichText", async () => {
  const t = "Hello @alice.com, check out this link: https://example.com";
  const rt = await getRichText(t);
  assertEquals(rt.text, t, "rich text has original text");
  assert(rt.facets!.length > 0, "rich text has facets");
});

In [ ]:
// deno-fmt-ignore-file
// === deno.json ===
// {
//   "imports": {
//     "hono": "https://deno.land/x/hono@v3.12.0/mod.ts"
//   },
//   "tasks": {
//     "start": "deno run --allow-net=cardyb.bsky.app,localhost --allow-env --allow-read src/index.ts"
//   }
// }

// src/utils/image.ts
// (Simplified placeholder implementation for image compression)
const compressPostImage = async (blob: Blob): Promise<{ blob: Blob }> => {
  // In a real implementation, this would involve actual image compression logic.
  // For simplicity, we're just returning the original blob.
  return { blob: blob };
};

// src/api/utils/misc.ts

// (Simplified followAbortSignal for demonstration)
const followAbortSignal = (signals: (AbortSignal | undefined)[]) => {
  const controller = new AbortController();
  const own = controller.signal;

  for (const signal of signals) {
    if (!signal) continue;

    if (signal.aborted) {
      controller.abort(signal.reason);
      break;
    }

    signal.addEventListener("abort", () => controller.abort(signal.reason), {
      signal: own,
    });
  }

  return own;
};

//src/index.ts

import { Hono } from "hono";
import { cors } from "hono/cors";

const app = new Hono();
app.use("/*", cors());

const LINK_PROXY_ENDPOINT = "https://cardyb.bsky.app/v1/extract";

interface LinkProxyResponse {
  error: string;
  likely_type: string;
  url: string;
  title: string;
  description: string;
  image: string;
}

interface LinkMeta {
  uri: string;
  title: string;
  description: string;
  thumb: Blob | undefined;
}

app.get("/getLinkMeta", async (c) => {
  const url = c.req.query("url");

  if (!url) {
    return c.json({ error: "Missing URL parameter" }, 400);
  }

  try {
    const response = await fetch(
      `${LINK_PROXY_ENDPOINT}?url=${encodeURIComponent(url)}`,
      {
        signal: followAbortSignal([c.req.signal, AbortSignal.timeout(5_000)]),
      }
    );

    if (!response.ok) {
      throw new Error(
        `Failed to contact proxy: response error ${response.status}`
      );
    }

    const data = (await response.json()) as LinkProxyResponse;

    if (data.error != "") {
      throw new Error(`Proxy error: ${data.error}`);
    }

    let thumb: Blob | undefined;
    if (data.image != "") {
      try {
        const thumbResponse = await fetch(data.image, {
          signal: followAbortSignal([
            c.req.signal,
            AbortSignal.timeout(10_000),
          ]),
        });

        if (!thumbResponse.ok) {
          throw new Error(
            `Failed to retrieve image: response error ${thumbResponse.status}`
          );
        }

        const blob = await thumbResponse.blob();
        const result = await compressPostImage(blob); // Using placeholder
        thumb = result.blob;
      } catch (thumbError) {
        console.error("Thumbnail processing error:", thumbError);
        // Not critical enough to abort request
      }
    }

    const meta: LinkMeta = {
      uri: url,
      title: data.title,
      description: data.description,
      thumb: thumb,
    };

    return c.json(meta);
  } catch (error) {
    console.error("Error fetching link meta:", error);
    return c.json({ error: String(error) }, 500); // or 502 for bad gateway
  }
});

// Deno.serve(app.fetch);


Hono {
  get: [Function (anonymous)],
  post: [Function (anonymous)],
  put: [Function (anonymous)],
  delete: [Function (anonymous)],
  options: [Function (anonymous)],
  patch: [Function (anonymous)],
  all: [Function (anonymous)],
  on: [Function (anonymous)],
  use: [Function (anonymous)],
  router: SmartRouter { name: "SmartRouter" },
  getPath: [Function: getPath],
  _basePath: "/",
  routes: [
    { path: "/*", method: "ALL", handler: [AsyncFunction: cors] },
    {
      path: "/getLinkMeta",
      method: "GET",
      handler: [AsyncFunction (anonymous)]
    }
  ],
  errorHandler: [Function: errorHandler],
  onError: [Function: onError],
  notFound: [Function: notFound],
  fetch: [Function: fetch],
  request: [Function: request],
  fire: [Function: fire]
}